expecting to import googleNet model

In [ ]:
from torch.optim import Adam
# import model from path
import torchvision.models as models
import torch.nn as nn
model = models.googlenet(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 4)
)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='CrossEntropyLoss',
    metrics=['accuracy']
)

In [ ]:
from path import X_train , y_train , X_test , y_test
import keras
import time
start = time.time()
EPOCHS =  25
checkpoint_filepath = 'checkpoints/model1/checkpoint_epoch_{epoch:02d}.model.keras'

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

history = model.fit(
        X_train,
        y_train,
        batch_size=16,
        epochs=EPOCHS,
        validation_split= 0.5,
        callbacks = [model_checkpoint_callback]
    )

end = time.time()
print("Temps total d'entraînement :", (end - start)/60, "minutes")

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test , batch_size  = 32)
print(f" Perte (loss) sur le test : {loss:.4f}")
print(f" Précision (accuracy) sur le test : {accuracy*100:.2f}%")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


print("\nGénération des prédictions pour la matrice de confusion...")
y_pred = model.predict(X_test, batch_size=32)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1) if len(y_test.shape) > 1 else y_test

cm = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Count'})
plt.title('Matrice de Confusion', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Classe Réelle', fontsize=12)
plt.xlabel('Classe Prédite', fontsize=12)
